In [113]:
import numpy as np
import pandas as pd
import time
import wrds
from matplotlib import pyplot as plt
import datetime
from dateutil.relativedelta import relativedelta
from scipy.interpolate import UnivariateSpline, splev, splrep
from scipy.optimize import minimize
from math import *
np.random.seed(6078)
import warnings
warnings.filterwarnings('ignore')
# from dask.multiprocessing import get
# import dask.dataframe as dd

In [114]:
opm = pd.read_csv('merged_data.csv', nrows=200000)
#opm['date'] = pd.to_datetime(opm['date'], format='%Y-%m-%d')
#opm['exdate'] = pd.to_datetime(opm['exdate'], format='%Y-%m-%d')
# opm = opm[(opm['exdate'] - opm['date'] == np.timedelta64(-8, 'D'))]
opm.rename(columns={'strike_price': 'STRIKE'}, inplace=True)
opm['STRIKE'] = opm['STRIKE']/1000

In [115]:
opm.head()

,secid,exdate,date,cp_flag,STRIKE,volume,open_interest,impl_volatility,opprc,moneyness,tte,close,spread,noi
0,210354,20190719,20190711,C,14.0,0,44,1.651489,8.550,0.622222,8,22.5,0.50,0
1,210354,20190719,20190711,C,18.0,1,10,0.859370,4.550,0.800000,8,22.5,0.10,0
2,210354,20190719,20190711,C,19.0,0,3,0.814534,3.600,0.844444,8,22.5,0.20,0
3,210354,20190719,20190711,C,20.0,0,39,0.419988,2.525,0.888889,8,22.5,0.35,0
4,210354,20190719,20190711,C,21.0,14,396,0.651550,1.800,0.933333,8,22.5,0.10,-40


In [116]:
# def clean_fairs(chain):
#     chain = chain.sort_values(["cp_flag", "STRIKE"]).reset_index()
#     phain = chain[chain['cp_flag'] == 'P']
#     chain = chain[chain['cp_flag'] == 'C']
#     chain['best_bid'] = -((-chain['best_bid']).cummax())
#     chain['best_offer'] = -((-chain['best_offer']).cummax())
#     phain['best_bid'] = ((phain['best_bid']).cummax())
#     phain['best_offer'] = ((phain['best_offer']).cummax())
#     prc = chain.PRC.max()
#     chain["CV"] = (chain["best_bid"] + chain["best_offer"]) / 2
#     phain["PV"] = (phain["best_bid"] + phain["best_offer"]) / 2
#     chain = chain[["STRIKE", "CV"]]
#     phain = phain[["STRIKE", "PV"]]
#     df = chain.merge(phain, on='STRIKE')
#     df["V"] = df["CV"] + df["PV"]# - (df['STRIKE'] - prc).abs()
#     return df

In [117]:
def clean_fairs(chain):
    chain = chain.sort_values(["cp_flag", "STRIKE"]).reset_index()
    phain = chain[chain['cp_flag'] == 'P']
    chain = chain[chain['cp_flag'] == 'C']
    chain['CV'] = -((-chain['opprc']).cummax())
    phain['PV'] = ((phain['opprc']).cummax())
    chain = chain[["STRIKE", "CV"]]
    phain = phain[["STRIKE", "PV"]]
    df = chain.merge(phain, on='STRIKE')
    df["V"] = df["CV"] + df["PV"]
    return df

In [118]:
def implied_probs(chain, s=0.1, M=1000):
    df = clean_fairs(chain)
    df =  df.drop_duplicates(subset='STRIKE', keep="first").reset_index()
    if len(df) < 5:
        return np.nan
    spl = UnivariateSpline(df['STRIKE'], df['V'], k=4, s=s*df.shape[0])
    df['Vhat'] = spl(df['STRIKE'])
    df['IP'] = np.maximum(spl.derivative().derivative()(df['STRIKE']), 0)
    df["PCDF"] = df["IP"].cumsum()
    df["PCDF"] = df["PCDF"] / df["PCDF"].max()
    return df

In [119]:
def implied_median(chain):
    df = implied_probs(chain)
    try:
        mid_idx = df[(df['PCDF'] > 0.5)].index[0]
        lb = df.loc[mid_idx-1, 'STRIKE']
        lbp = df.loc[mid_idx-1, 'PCDF']
        ub = df.loc[mid_idx, 'STRIKE']
    except:
        return np.nan
    
    ubp = df.loc[mid_idx, 'PCDF']
    slope = (ubp - lbp) / (ub - lb)
    res = (0.5 - lbp) / slope + lb
    return res

def implied_move(chain):
    return implied_median(chain) / chain.PRC.max() - 1

In [120]:
# chain = opm[opm['secid'] == 106367]
# chain = chain[chain['exdate'] == '2018-01-15']
# df = implied_probs(chain)
# print(implied_median(chain))

In [121]:
opm.columns

Index(['secid', 'exdate', 'date', 'cp_flag', 'STRIKE', 'volume',
       'open_interest', 'impl_volatility', 'opprc', 'moneyness', 'tte',
       'close', 'spread', 'noi'],
      dtype='object')

In [122]:
# df.plot(x='STRIKE', y='Vhat')
# df.plot(x='STRIKE', y='IP')
# df.plot(x='STRIKE', y='PCDF')

In [123]:
(df['IP'] * df['STRIKE']).sum() / df['IP'].sum()

372.30307126506295

In [124]:
opm.groupby(["secid", "exdate"]).apply(implied_median)

secid   exdate  
8170    20180615          NaN
        20190215          NaN
        20191115          NaN
100862  20181221          NaN
        20191220          NaN
                      ...    
213731  20190906    91.806615
213761  20191115          NaN
213776  20190920    28.744683
        20191213    24.137452
213814  20191115          NaN
Length: 2541, dtype: float64

In [126]:
opm_app_mean = opm.groupby(["secid", "exdate"]).apply(implied_median)

In [127]:
opm_app_mean.to_csv("opm_app_mean.csv")